In [26]:
import numpy as np
import scipy.optimize as opt
import math
import pandas as pd

# 螺线方程 r = b*theta
b = 0.55 / (2 * np.pi)

# 求解当前龙头走过的距离
def calcule_distance(v_dragon_head, time):
    return v_dragon_head * time

# 求解阿基米德螺线弧长
def calcule_arc_length(theta):
    return 0.5 * b * (theta * np.sqrt(1 + theta ** 2) + np.arcsinh(theta))

# 定义龙头目标函数
def equation1(theta, time, v_dragon_head):
    distance = calcule_distance(v_dragon_head, time)
    return calcule_arc_length(32 * np.pi) - 0.5 * b * (theta * np.sqrt(1 + theta ** 2) + np.arcsinh(theta)) - distance

# 求解龙头在阿基米德螺线上的极角
def calculate_theta(time, v_dragon_head):
    initial_guess = 0
    theta_solution, = opt.fsolve(equation1, initial_guess, args=(time, v_dragon_head))
    return theta_solution

# 求解位置
def calculate_position(theta):
    if theta > 32 * np.pi:
        x,y = 0, 0
    else:
        x = b * theta * np.cos(theta)
        y = b * theta * np.sin(theta)
    return x, y

# 定义龙身目标函数
def equation2(theta1, theta2, l):
    return (b*theta1)**2 + (b*theta2)**2 - 2 * b * b * theta1 * theta2 * np.cos(theta2 - theta1) - l**2

# 递归求解龙身的极角
def calculate_body_theta(theta, l):
    initial_guess = theta + 0.09  # 初始猜测值略大于 theta
    theta_solution, = opt.fsolve(equation2, initial_guess, args=(theta, l))
    return theta_solution

# 阿基米德螺线导函数
def archimedean_spiral_derivative(theta):
    numerator = b * np.sin(theta) + b * theta * np.cos(theta)
    denominator = b * np.cos(theta) - b * theta * np.sin(theta)
    return numerator / denominator

# 求theta 和theta1的两点的方程
def line(theta1, theta2):
    x1, y1 = calculate_position(theta1)
    x2, y2 = calculate_position(theta2)
    return  (y2 - y1) / (x2 - x1)

# 计算两直线夹角
def calculate_angle(m1, m2):
    if m1 == m2:
        return 0  # 平行直线夹角为 0 度
    elif m1 * m2 == -1:
        return np.pi / 2  # 垂直直线夹角为 90 度

    tan_theta = abs((m1 - m2) / (1 + m1 * m2))
    theta_radians = math.atan(tan_theta)
    return theta_radians

# 求解龙身速度
def calculate_body_velocity(theta1, theta2, v_dragon_head):
    if theta1 >32*np.pi or theta2 > 32 * np.pi:
        return 0
    k1 = archimedean_spiral_derivative(theta1)
    k2 = archimedean_spiral_derivative(theta2)
    k3 = line(theta1, theta2)
    beta1 = calculate_angle(k1, k3)
    beta2 = calculate_angle(k2, k3)
    v_body = v_dragon_head * np.cos(beta1) / np.cos(beta2)
    return v_body


In [27]:
#使用示例
time = 428.9817617653601
v_dragon_head = 1
l = 2.86
theta1 = calculate_theta(time, v_dragon_head)
print(f"当前龙头极角： {theta1}")
x1, y1 = calculate_position(theta1)
print(f"龙头位置: x = {x1}, y = {y1}")

theta2 = calculate_body_theta(theta1, 0)
print(f"龙头后把手极角： {theta2}")
x2, y2 = calculate_position(theta2)
print(f"龙头后把手位置: x = {x2}, y = {y2}")
v_body = calculate_body_velocity(theta1,theta2, v_dragon_head)
print(f"龙头后把手速度: {v_body}")
theta3 = calculate_body_theta(theta2, l)
print(f"龙身极角： {theta3}")
x3, y3 = calculate_position(theta3)
print(f"龙身位置: x = {x3}, y = {y3}")
v_body = calculate_body_velocity(theta2,theta3, v_dragon_head)
print(f"龙身速度: {v_body}")

当前龙头极角： 17.517688025530326
龙头位置: x = 0.3629004176825358, y = -1.489853611322843
龙头后把手极角： 17.51768803808484
龙头后把手位置: x = 0.36290043664700844, y = -1.489853607834547
龙头后把手速度: 0.9999999999999999
龙身极角： 37.42797233471707
龙身位置: x = 3.156571725804316, y = -0.8774805776167401
龙身速度: 2.034043700869193


In [23]:
# 初始化结果列表
results_position = []  # 记录位置 (x, y)
results_velocity = []  # 记录速度

# 计算每秒钟的位置和速度
for t in range(301):
    time_result_position = []
    time_result_velocity = []

    # 计算龙头位置和速度
    theta_head = calculate_theta(t, 1)
    x_head, y_head = calculate_position(theta_head)
    time_result_position.extend([x_head, y_head])
    time_result_velocity.append(1)

    prev_theta = theta_head
    pre_v = 1

    for i in range(1, 224):  # 计算龙身位置和速度
        length = 2.86 if i == 1 else 1.65
        theta_body = calculate_body_theta(prev_theta, length)
        x_body, y_body = calculate_position(theta_body)
        v_body = calculate_body_velocity(prev_theta, theta_body, pre_v)
        time_result_position.extend([x_body, y_body])
        time_result_velocity.append(v_body)
        prev_theta = theta_body
        pre_v = v_body

    # 保存当前时间点的结果
    results_position.append(time_result_position)
    results_velocity.append(time_result_velocity)

# 转换为DataFrame
position_columns = []
velocity_columns = []
for i in range(224):
    position_columns.extend([f'x_{i}', f'y_{i}'])
    velocity_columns.append(f'v_{i}')

df_position = pd.DataFrame(results_position, columns=position_columns)
df_velocity = pd.DataFrame(results_velocity, columns=velocity_columns)


df_velocity = df_velocity.T
df_position = df_position.T
#保存到Excel文件
with pd.ExcelWriter('result1.xlsx') as writer:
    df_position.to_excel(writer, sheet_name='位置', index=False)
    df_velocity.to_excel(writer, sheet_name='速度', index=False)

print("结果已保存到 result1.xlsx 文件中")

结果已保存到 result1.xlsx 文件中


In [24]:
import pandas as pd

# 读取Excel文件中的工作表
df_position = pd.read_excel("D:\\code\\vscode\\数学建模模型\\2024A\\result1.xlsx", sheet_name="位置")
df_velocity = pd.read_excel("D:\\code\\vscode\\数学建模模型\\2024A\\result1.xlsx", sheet_name="速度")

# 生成新索引
index = ["龙头x (m)", "龙头y (m)"]
for i in range(1, 222):
    index.append(f'第{i}节龙身x (m)')
    index.append(f'第{i}节龙身y (m)')
index.append("龙尾x (m)")
index.append("龙尾y (m)")
index.append("龙尾（后）x (m)")
index.append("龙尾（后）y (m)")

index1 = ["龙头 (m/s)"]
for i in range(1, 222):
    index1.append(f'第{i}节龙身 (m/s)')
index1.append("龙尾 (m/s)")
index1.append("龙尾（后）(m/s)")

# 生成时间点索引

time_points = [f"{i} s" for i in range(301)]
# 设置列名为时间点
df_position.columns = time_points
df_velocity.columns = time_points

# 将索引设置为新生成的索引
df_position.set_index(pd.Index(index), inplace=True)
df_velocity.set_index(pd.Index(index1), inplace=True)

# 保存修改后的数据到新的Excel文件
with pd.ExcelWriter('D:\\code\\vscode\\数学建模模型\\2024A\\result_modified.xlsx') as writer:
    df_position.to_excel(writer, sheet_name='位置', index=True)
    df_velocity.to_excel(writer, sheet_name='速度', index=True)

print("索引已修改并保存到 result_modified.xlsx 文件中")

索引已修改并保存到 result_modified.xlsx 文件中


In [25]:
# 第二问持续到 t = 428.9817617653601
t = 428.9817617653601
time_result_position = []
time_result_velocity = []

# 计算龙头位置和速度
theta_head = calculate_theta(t, 1)
print(f"theta_head: {theta_head}")
x_head, y_head = calculate_position(theta_head)
print(f"x_head: {x_head}, y_head: {y_head}")
time_result_position.append([x_head, y_head])
time_result_velocity.append(1)
print(time_result_position)
prev_theta = theta_head
pre_v = 1

for i in range(1, 224):  # 计算龙身位置和速度

    length = 2.86 if i == 1 else 1.65
    theta_body = calculate_body_theta(prev_theta, length)
    x_body, y_body = calculate_position(theta_body)
    v_body = calculate_body_velocity(prev_theta, theta_body, pre_v)
    time_result_position.append([x_body, y_body])
    time_result_velocity.append(v_body)
    prev_theta = theta_body
    pre_v = v_body

# 创建行索引
index_names = ["龙头"] + [f"第{i}节龙身" for i in range(1, 222)] + ["龙尾", "龙尾（后）"]

# 创建数据框
df_position = pd.DataFrame(time_result_position, columns=["横坐标x (m)", "纵坐标y (m)"], index=index_names)
df_velocity = pd.DataFrame(time_result_velocity, columns=["速度 (m/s)"], index=index_names)

# 将位置和速度合并到一个数据框中
result_df = pd.concat([df_position, df_velocity], axis=1)

# 保存到 Excel 文件
result_df.to_excel("result2.xlsx")

print("结果已保存到 result2.xlsx")

theta_head: 17.517688025530326
x_head: 0.3629004176825358, y_head: -1.489853611322843
[[0.3629004176825358, -1.489853611322843]]
结果已保存到 result2.xlsx


D:\Program Files\Anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
